In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 84 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595728 sha256=7d456b9fd88400a0762294b8b33e0d73ee6dde3e5d70c66b43d10ab84b553bdc
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 10.0 MB/s 
     |████████████████████████████████| 754 kB 10.2 MB/s 
     |████████████████████████████████| 3.0 MB 44.2 MB/s 
     |████████████████████████████████| 895 kB 55.0 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-envyoqy7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-envyoqy7
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=47edcf9703cd0041b0f4c70171d4b0521e936b86ae57af0a154b380c2a67d32a
  Stored in directory: /tmp/pip-ephem-wheel-cache-x8dow60t/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer


from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
cd /content/drive/MyDrive/data/pjt2

/content/drive/MyDrive/data/pjt2


In [17]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('데이터_추가2_전처리(불용어추가).csv')
train_df = train_df.dropna()

In [18]:
         train_dataset = []
for sen, label in zip(train_df['new_title3'], train_df['topic_idx']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)
# kobert 트레인 할때 input 방식

In [19]:
train_dataset[:5]

[[' 인천 핀란드 항공기 결항 휴가 여행객 분통', '4'],
 [' 실리콘밸리 넘어서다 구글 조원 들이다 미국 전역 거점', '4'],
 [' 이란 외무 긴장 완화 해결 미국 경제 전쟁 멈추다', '4'],
 [' 클린턴 측근 한국 기업 특수 관계 조명 공과 맞다 물리다', '4'],
 [' 시진핑 트럼프 중미 무역 협상 조속 타결 희망', '4']]

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [21]:
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5
# 파라미터 설정

In [22]:
bertmodel, vocab = get_pytorch_kobert_model() ##kober 모델 정보 가지고오기(단어)

using cached model
using cached model


In [23]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

using cached model


In [24]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

In [25]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
model = BERTClassifier(bertmodel, dr_rate=0.4).to(device)


#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
for e in range(num_epochs):
    train_acc = 0.0
    # test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1669 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8486253023147583 train acc 0.3125
epoch 1 batch id 201 loss 1.192788004875183 train acc 0.36116293532338306
epoch 1 batch id 401 loss 1.0634326934814453 train acc 0.5707605985037406
epoch 1 batch id 601 loss 0.7575345635414124 train acc 0.6497504159733777
epoch 1 batch id 801 loss 0.37633195519447327 train acc 0.6926888264669163
epoch 1 batch id 1001 loss 0.1573471575975418 train acc 0.7351710789210789
epoch 1 batch id 1201 loss 0.4890333116054535 train acc 0.7630099916736053
epoch 1 batch id 1401 loss 0.3592931926250458 train acc 0.7734653818700928
epoch 1 batch id 1601 loss 0.0020026692654937506 train acc 0.7910290443472829
epoch 1 train acc 0.7936076992210904


  0%|          | 0/1669 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 7.066936492919922 train acc 0.0
epoch 2 batch id 201 loss 0.5462185144424438 train acc 0.775342039800995
epoch 2 batch id 401 loss 0.7500874996185303 train acc 0.8050187032418953
epoch 2 batch id 601 loss 0.736149787902832 train acc 0.8194155574043261
epoch 2 batch id 801 loss 0.4140304625034332 train acc 0.8301732209737828
epoch 2 batch id 1001 loss 0.16883951425552368 train acc 0.8528346653346653
epoch 2 batch id 1201 loss 0.28067171573638916 train acc 0.8668037052456287
epoch 2 batch id 1401 loss 0.2554159462451935 train acc 0.8689775160599572
epoch 2 batch id 1601 loss 0.0013550869189202785 train acc 0.8776936289818863
epoch 2 train acc 0.8820775913720791


  0%|          | 0/1669 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 3.48960542678833 train acc 0.28125
epoch 3 batch id 201 loss 0.3836294710636139 train acc 0.8673818407960199
epoch 3 batch id 401 loss 0.704957127571106 train acc 0.8738310473815462
epoch 3 batch id 601 loss 0.5485679507255554 train acc 0.8795237104825291
epoch 3 batch id 801 loss 0.320607453584671 train acc 0.8854166666666666
epoch 3 batch id 1001 loss 0.11771493405103683 train acc 0.9011925574425574
epoch 3 batch id 1201 loss 0.1539224088191986 train acc 0.9113759367194005
epoch 3 batch id 1401 loss 0.10882820934057236 train acc 0.9133877587437544
epoch 3 batch id 1601 loss 0.002429818268865347 train acc 0.9191130543410369
epoch 3 train acc 0.9219779808268425


  0%|          | 0/1669 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 1.6790341138839722 train acc 0.65625
epoch 4 batch id 201 loss 0.2361600399017334 train acc 0.9059390547263682
epoch 4 batch id 401 loss 0.6486193537712097 train acc 0.911783042394015
epoch 4 batch id 601 loss 0.29734522104263306 train acc 0.9161293677204659
epoch 4 batch id 801 loss 0.18358218669891357 train acc 0.9199828339575531
epoch 4 batch id 1001 loss 0.06014807149767876 train acc 0.9313811188811189
epoch 4 batch id 1201 loss 0.07506518065929413 train acc 0.9382545795170691
epoch 4 batch id 1401 loss 0.054054610431194305 train acc 0.9388160242683797
epoch 4 batch id 1601 loss 0.05206604674458504 train acc 0.9416965958775765
epoch 4 train acc 0.9434281136843202


In [15]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

using cached model


In [16]:
test_df = pd.read_csv('test_data.csv')
sub_df = pd.read_csv('sample_submission.csv')

In [17]:
pre = []

for i in tqdm(test_df['title']):
  pre.append(predict(i))

  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 9131/9131 [49:48<00:00,  3.06it/s]


In [18]:
sub_df['topic_idx'] = pre
sub_df.to_csv('predict_kobert.csv', index=False)